This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace.

In [ ]:
!pip install chromadb
!pip install langchain
!pip install openai==0.28
!pip install gradio
!pip install tiktoken
!pip install gradio
!pip install -U langchain-openai

  Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.26.0
    Uninstalling openai-1.26.0:
      Successfully uninstalled openai-1.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.6 requires openai<2.0.0,>=1.24.0, but you have openai 0.28.0 which is incompatible.
  Using cached openai-1.26.0-py3-none-any.whl (314 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [ ]:
from langchain.llms import OpenAI
from langchain import PromptTemplate
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.chat_models import ChatOpenAI
import io
import gradio as gr
from google.colab import userdata
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_API_KEY')
model_name = "gpt-4-turbo"
temperature=0.1

In [ ]:
loader = CSVLoader(file_path='beers.csv')
docs = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

In [ ]:
embeddings = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
db = Chroma.from_documents(split_docs, embeddings)

In [ ]:
# This is a base set
questions = [
                "Do you like hoppy or malty beer?",
                "What are the most important flavors for you in a beer?",
                "What color beers do you usually drink?",
                "How much carbonation do you like?",
                "What else about the beer is important to you?",
            ]

In [ ]:
query = """
    You are an cicerone with great taste and love to recommend new beers to beer drinkers.
    You follow human instructions and provides new beer recommendations for a beer drinker based on the available beer entries provided
    and beer drinker's persona derived from their answers to questions.

Personal Questions and Answers:
Ask the buyer 5 questions
You:  Do you like hoppy or malty beer?
Buyer: {answer1}
You: What are the most important flavors for you in a beer?
Buyer: {answer2}
You: What color beers do you usually drink?
Buyer: {answer3}
You: How much carbonation do you like?
Buyer: {answer4}
You: What else about the beer is important to you?
Buyer: {answer5}

Please write persuasive recommendations for the 5 most relevant beers explicitly tied to buyer preferences
- Please format generated recommendations "Based on your preferences, I recommend the following beers:" followed by the generated recommedation
- Please list out the beer name and full description in the generated recommendations
- If you cannot generate beer name and description, please say I cannot recommend a beer.
- Please address recommedations to the beer drinker in the first person.
- Do not change or modify beer drinker preferences


"""



In [ ]:
def do_recommendation_5(answer1, answer2, answer3, answer4, answer5):

    retriever = db.as_retriever(search_kwargs={"k": 10})
    str_prompt = query.format(answer1=answer1, answer2=answer2, answer3=answer3, answer4=answer4, answer5=answer5)
    llm = ChatOpenAI(model_name='gpt-4')
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    llm_response = rag(str_prompt)
    return llm_response["result"]


In [ ]:
import gradio as gr
with gr.Blocks() as demo:
    with gr.Row():
        vector_query1 = gr.Textbox(value="", show_label=True, label=questions[0])
    with gr.Row():
        vector_query2 = gr.Textbox(value="", show_label=True, label=questions[1])
    with gr.Row():
        vector_query3 = gr.Textbox(value="", show_label=True, label=questions[2])
    with gr.Row():
        vector_query4 = gr.Textbox(value="", show_label=True, label=questions[3])
    with gr.Row():
        vector_query5 = gr.Textbox(value="", show_label=True, label=questions[4])
    with gr.Row():
        b1 = gr.Button("Submit")
    with gr.Row():
        result = gr.TextArea(value="This is where the results go", elem_id="result", label="Your results")

    b1.click(do_recommendation_5, inputs=[vector_query1,vector_query2,vector_query3,vector_query4,vector_query5], outputs=result)

demo.launch(server_name="0.0.0.0", inline=False)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5d54cd415d600c0d04.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
